# Deep Learning model for predicting the number of corners in a football match

### Import starting data from excel
#### Starting data contains the premier league football games from the previous 7 years and has variables like, odds for the match, the strength of the team, avarage corner/free kick/goals/etc. on the last 10 match for both home and away team

In [2]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_bc050f45180b4c1bb8886542b85a43fe = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='xJ2wPgGOIN_BHKjLVYpM3__wCyHK6s98gBYL2bVzQcOB',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_bc050f45180b4c1bb8886542b85a43fe.get_object(Bucket='corner-donotdelete-pr-vmvgllkodfzoji',Key='Modelling_input.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_0 = pd.read_excel(body)
df_data_0.head()


ID        Date     HomeTeam     AwayTeam  B365H  \
0      27/12/2019WolvesMan City  27/12/2019       Wolves     Man City   8.50   
1   26/12/2019TottenhamBrighton  26/12/2019    Tottenham     Brighton   1.53   
2  26/12/2019Aston VillaNorwich  26/12/2019  Aston Villa      Norwich   2.10   
3  26/12/2019BournemouthArsenal  26/12/2019  Bournemouth      Arsenal   3.50   
4  26/12/2019ChelseaSouthampton  26/12/2019      Chelsea  Southampton   1.33   

   B365D  B365A  B365>2.5  B365<2.5  B365 H-A  ...  \
0   5.00   1.36      1.57      2.37      7.14  ...   
1   4.33   6.00      1.61      2.30      4.47  ...   
2   3.75   3.30      1.53      2.50      1.20  ...   
3   3.90   1.95      1.57      2.37      1.55  ...   
4   5.25   9.00      1.50      2.62      7.67  ...   

   AwayTeam_10M_avaragesum corner  AwayTeam_10M_avarageCorner/Free kick %  \
0                            12.5                                1.266569   
1                             9.9                                0.469910   
2                            12.9                                0.636632   
3                            13.5                                0.715851   
4                            11.8                                0.477563   

   AwayTeam_10M_avarageShots/Corner  AwayTeam_10M_avarageGoals/Corner  \
0                          2.444544                          0.335451   
1                          2.614008                          0.258175   
2                          3.785079                          0.326587   
3                          1.675000                          0.190278   
4                          2.394167                          0.213333   

   AwayTeam_10M_avarageYellow  AwayTeam_10M_avaragesum yellow  \
0                         2.0                             3.1   
1                         1.7                             2.9   
2                         2.1                             3.9   
3                         2.1                             4.1   
4                         1.3                             3.3   

   AwayTeam_10M_avarageRed  AwayTeam_10M_avaragesum red  corner_9_5  \
0                      0.1                          0.1           0   
1                      0.1                          0.2           1   
2                      0.0                          0.1           1   
3                      0.0                          0.0           1   
4                      0.1                          0.1           0   

   corner_10_5  
0            0  
1            1  
2            1  
3            1  
4            0  

[5 rows x 58 columns]

In [3]:
df_data_0.shape

(2449, 58)

#### Drop unnecessary columns

In [4]:
X=df_data_0.drop(columns=['ID','Date','HomeTeam','AwayTeam','corner_9_5','corner_10_5'])
X.head()

B365H  B365D  B365A  B365>2.5  B365<2.5  B365 H-A  Place home  Place away  \
0   8.50   5.00   1.36      1.57      2.37      7.14           7           3   
1   1.53   4.33   6.00      1.61      2.30      4.47           6          14   
2   2.10   3.75   3.30      1.53      2.50      1.20          16          20   
3   3.50   3.90   1.95      1.57      2.37      1.55          17          13   
4   1.33   5.25   9.00      1.50      2.62      7.67           4          12   

   Place diff  Place sum  ...  AwayTeam_10M_avaragesum free kick  \
0           4         10  ...                               18.4   
1           8         20  ...                               19.0   
2           4         36  ...                               20.6   
3           4         30  ...                               23.3   
4           8         16  ...                               23.6   

   AwayTeam_10M_avarageCorner  AwayTeam_10M_avaragesum corner  \
0                        10.0                            12.5   
1                         4.9                             9.9   
2                         5.2                            12.9   
3                         7.3                            13.5   
4                         5.2                            11.8   

   AwayTeam_10M_avarageCorner/Free kick %  AwayTeam_10M_avarageShots/Corner  \
0                                1.266569                          2.444544   
1                                0.469910                          2.614008   
2                                0.636632                          3.785079   
3                                0.715851                          1.675000   
4                                0.477563                          2.394167   

   AwayTeam_10M_avarageGoals/Corner  AwayTeam_10M_avarageYellow  \
0                          0.335451                         2.0   
1                          0.258175                         1.7   
2                          0.326587                         2.1   
3                          0.190278                         2.1   
4                          0.213333                         1.3   

   AwayTeam_10M_avaragesum yellow  AwayTeam_10M_avarageRed  \
0                             3.1                      0.1   
1                             2.9                      0.1   
2                             3.9                      0.0   
3                             4.1                      0.0   
4                             3.3                      0.1   

   AwayTeam_10M_avaragesum red  
0                          0.1  
1                          0.2  
2                          0.1  
3                          0.0  
4                          0.1  

[5 rows x 52 columns]

In [5]:
Y=df_data_0[['corner_9_5']].values
Y

array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

### Prepare data

In [6]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import numpy as np

In [7]:
scaler = StandardScaler()
X_std=scaler.fit_transform(X)
X_std.shape

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


(2449, 52)

In [8]:
from sklearn.model_selection import train_test_split
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X_std, Y, test_size=0.3, random_state=3)


### Model parameters

In [9]:
import tensorflow as tf
from tensorflow import keras


In [10]:
model = keras.Sequential([
    keras.layers.Dense(4, activation='relu'),
    keras.layers.Dense(4, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
    
])

In [11]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
model.fit(X_trainset, y_trainset, epochs=20)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
1714/1714 [==============================] - 41s 24ms/sample - loss: 0.6918 - acc: 0.5578
Epoch 2/20
1714/1714 [==============================] - 48s 28ms/sample - loss: 0.6855 - acc: 0.5916
Epoch 3/20
1714/1714 [==============================] - 47s 27ms/sample - loss: 0.6809 - acc: 0.5963
Epoch 4/20
1714/1714 [==============================] - 47s 27ms/sample - loss: 0.6768 - acc: 0.5951
Epoch 5/20
1714/1714 [==============================] - 49s 28ms/sample - loss: 0.6728 - acc: 0.5951
Epoch 6/20
1714/1714 [==============================] - 48s 28ms/sample - loss: 0.6701 - acc: 0.5951
Epoch 7/20
1714/1714 [==============================] - 48s 28ms/sample - loss: 0.6682 - acc: 0.5951
Epoch 8/20
1714/1714 [==============================] - 43s 25ms/sample - loss: 0.6667 - acc: 0.5951
Epoch 9/20
1714/1714 [==============================] - 41s 24ms/sample - loss: 0.6655 - acc: 0.5951
Epoch 10/20
1714/1714 [====================

### Results

In [13]:
test_loss, test_acc = model.evaluate(X_testset, y_testset, verbose=2)

print('\nTest accuracy:', test_acc)

 - 8s - loss: 0.6706 - acc: 0.6150

Test accuracy: 0.614966


#### 61% seems like a good result, betting sites usually pay 1,83 odd for guessing it right, so the expected return: 1,83 * 0,61 + 0 * 0,39= 1,1163, so if the model works in real life it could generate 11,63% profit for each match in avarage
#### The result of other models are in the 'Modelling results.xlsx' file
#### Overfitting was prevented with early stopping (limiting number of epochs)

### Try to prepare the model with less variable (with variables that are easy to acces)

In [14]:
X=df_data_0[['B365H','B365D','B365A','B365>2.5','B365<2.5','B365 H-A','Place home','Place away','Place diff','Place sum']].values
X[0:1]

array([[ 8.5 ,  5.  ,  1.36,  1.57,  2.37,  7.14,  7.  ,  3.  ,  4.  ,
        10.  ]])

In [15]:
scaler = StandardScaler()
X_std=scaler.fit_transform(X)
X_std.shape

(2449, 10)

In [16]:
from sklearn.model_selection import train_test_split
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X_std, Y, test_size=0.3, random_state=3)
X_trainset.shape

(1714, 10)

In [45]:
model = keras.Sequential([
    keras.layers.Dense(4, input_shape=(10,)  ,activation='relu'),
    keras.layers.Dense(4, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
    
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [46]:
model.fit(X_trainset, y_trainset, epochs=20)

Epoch 1/20
1714/1714 [==============================] - 43s 25ms/sample - loss: 0.7557 - acc: 0.4638
Epoch 2/20
1714/1714 [==============================] - 39s 23ms/sample - loss: 0.7121 - acc: 0.5047
Epoch 3/20
1714/1714 [==============================] - 43s 25ms/sample - loss: 0.6938 - acc: 0.5449
Epoch 4/20
1714/1714 [==============================] - 44s 26ms/sample - loss: 0.6852 - acc: 0.5630
Epoch 5/20
1714/1714 [==============================] - 40s 23ms/sample - loss: 0.6803 - acc: 0.5735
Epoch 6/20
1714/1714 [==============================] - 44s 26ms/sample - loss: 0.6774 - acc: 0.5770
Epoch 7/20
1714/1714 [==============================] - 42s 24ms/sample - loss: 0.6755 - acc: 0.5846
Epoch 8/20
1714/1714 [==============================] - 44s 26ms/sample - loss: 0.6742 - acc: 0.5904
Epoch 9/20
1714/1714 [==============================] - 40s 23ms/sample - loss: 0.6734 - acc: 0.5922
Epoch 10/20
1714/1714 [==============================] - 42s 24ms/sample - loss: 0.6727 - a

In [47]:
test_loss, test_acc = model.evaluate(X_testset, y_testset, verbose=2)

print('\nTest accuracy:', test_acc)

 - 7s - loss: 0.6699 - acc: 0.6136

Test accuracy: 0.61360544


#### Seems like we can keep the performance of the model with less variable

In [48]:
model.predict(X_testset)[0:5]

array([[0.5534794 ],
       [0.61737925],
       [0.5452771 ],
       [0.58987015],
       [0.54971045]], dtype=float32)

#### Check the performace on the last 100 football match, to see how well the model performs in the recent matches

In [49]:
X_100=X_std[0:100]
Y_100=Y[0:100]
X_100.shape

(100, 10)

In [50]:
test_loss, test_acc = model.evaluate(X_100, Y_100, verbose=2)

print('\nTest accuracy:', test_acc)

 - 1s - loss: 0.6839 - acc: 0.5900

Test accuracy: 0.59


#### A bit lower accuracy (59%), but above 54,6 %, so hopefully we can still use it to generate profit

### Saving the model

In [51]:
model.save( "corner9-model.h5" )

In [52]:
!tar -zcvf corner9-model.tgz corner9-model.h5

corner9-model.h5


In [53]:
!ls -l

total 120
-rw-r----- 1 dsxuser dsxuser 28416 Jan 19 09:23 corner_9_5_-model.h5
-rw-r----- 1 dsxuser dsxuser 28416 Jan 19 09:24 corner-9-5-model.h5
-rw-r----- 1 dsxuser dsxuser 28416 Jan 19 10:04 corner9-model.h5
-rw-r----- 1 dsxuser dsxuser  3257 Jan 19 10:04 corner9-model.tgz
-rw-r----- 1 dsxuser dsxuser 28416 Jan 19 09:24 message-classification-model.h5
-rw-r----- 1 dsxuser dsxuser  3253 Jan 19 09:25 message-classification-model.tgz


In [54]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
wml_credentials = {
    "apikey"    : "vTAQj-41jEpVVQdFkoVDNtGkjoQUGm4RN5FVgFJvJi5K",
    "instance_id" : "c8c21203-8d35-475b-b52f-0c376e8c1ea4",
    "url"    : "https://us-south.ml.cloud.ibm.com"
}
client = WatsonMachineLearningAPIClient( wml_credentials )

In [57]:
metadata = {
    client.repository.ModelMetaNames.NAME: "keras model",
    client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow",
    client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.13",
    client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES: [{'name':'keras', 'version': '2.1.6'}]}
model_details = client.repository.store_model( model="corner9-model.tgz", meta_props=metadata )

In [58]:
model_id = model_details["metadata"]["guid"]
deployment_details = client.deployments.create( artifact_uid=model_id, name="Keras deployment" )



#######################################################################################

Synchronous deployment creation for uid: 'd32d367d-9a34-4603-809e-ee945c25b89c' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS..
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='edee7088-f4ba-452c-9d9f-dc0c5a5d827a'
------------------------------------------------------------------------------------------------




In [59]:
deployment_details

{'metadata': {'guid': 'edee7088-f4ba-452c-9d9f-dc0c5a5d827a',
  'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/c8c21203-8d35-475b-b52f-0c376e8c1ea4/deployments/edee7088-f4ba-452c-9d9f-dc0c5a5d827a',
  'created_at': '2020-01-19T10:07:49.721Z',
  'modified_at': '2020-01-19T10:07:49.886Z'},
 'entity': {'runtime_environment': 'None Provided',
  'name': 'Keras deployment',
  'scoring_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/c8c21203-8d35-475b-b52f-0c376e8c1ea4/deployments/edee7088-f4ba-452c-9d9f-dc0c5a5d827a/online',
  'deployable_asset': {'name': 'keras model',
   'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/c8c21203-8d35-475b-b52f-0c376e8c1ea4/published_models/d32d367d-9a34-4603-809e-ee945c25b89c',
   'guid': 'd32d367d-9a34-4603-809e-ee945c25b89c',
   'created_at': '2020-01-19T10:07:49.692Z',
   'type': 'model'},
  'description': 'Description of deployment',
  'status_details': {'status': 'DEPLOY_SUCCESS'},
  'model_type': 'tensorflow-1.13',
  's

In [64]:
model_endpoint_url='https://us-south.ml.cloud.ibm.com/v3/wml_instances/c8c21203-8d35-475b-b52f-0c376e8c1ea4/deployments/edee7088-f4ba-452c-9d9f-dc0c5a5d827a/online'
payload={ "values" : X_std[0:5].tolist() }  
client.deployments.score( model_endpoint_url, payload )

{'fields': ['prediction', 'prediction_classes', 'probability'],
 'values': [[[0.657500684261322], [1], [0.657500684261322]],
  [[0.6473328471183777], [1], [0.6473328471183777]],
  [[0.5883569121360779], [1], [0.5883569121360779]],
  [[0.5870886445045471], [1], [0.5870886445045471]],
  [[0.6874839663505554], [1], [0.6874839663505554]]]}

In [65]:
model.predict(X_std)[0:5]

array([[0.6575007 ],
       [0.6473328 ],
       [0.5883569 ],
       [0.58708864],
       [0.6874839 ]], dtype=float32)